# Chemical Space Network 

## 1. Import RDKit, Networkx, and other libraries

In [1]:
# RDKit stuff
from rdkit import Chem
from rdkit import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import rdFMCS
from rdkit import DataStructs
from rdkit.Chem import rdmolops

# numpy
import numpy as np

# pandas
import pandas as pd

# networkx
import networkx as nx

# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# Print versions of libraries used
print('RDKit version: ',rdBase.rdkitVersion)
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)
print('Networkx version',nx.__version__)
print('MatplotLib version:', mpl.__version__)

RDKit version:  2023.09.6
Numpy version: 1.24.4
Pandas version: 2.0.1
Networkx version 2.8.8
MatplotLib version: 3.7.1


## 2. Load Data


In [3]:
full_df = pd.read_excel("../Data/IDPI_DataSet_Final_JPR.xlsx")


# 3. Data Preparation and Checks



In [4]:
# Check for presence of disconnected SMILES notation via string matching 
df1 = full_df[~full_df['nucleophile SMILES'].str.contains("\.")]
len(df1)==len(full_df)

True

In [5]:

smis = df1['nucleophile SMILES'].tolist()
num_frags = []
for smi in smis:
    mol = Chem.MolFromSmiles(smi)
    num_frags.append(len(Chem.GetMolFrags(mol))) # returns number of fragments

In [6]:
# now check that all molecules have only one fragment
all(frag == 1 for frag in num_frags)

True

In [7]:
# Double check that all SMILES are unique (different) compounds
# To be on the safe side, we can parse the SMILES as RDKit mol objects
# then write out canonical smiles and check

smis = np.unique(df1['nucleophile SMILES'].tolist())
rdkit_can_smiles = []
for smi in smis:
    mol = Chem.MolFromSmiles(smi)       
    rdkit_can_smiles.append(Chem.MolToSmiles(mol, canonical=True)) # default is true
    
set_rdkit_can_smiles = set(rdkit_can_smiles)
len(set_rdkit_can_smiles) == len(rdkit_can_smiles)

True

In [8]:
# We'll use the original SMILES as unique dictionary keys, so we should verify that the
# ChEMBL SMILES are unique strings too.
set_smis = set(smis)
len(set_smis) == len(smis)

True

## 4. Compile Node data

In [9]:
# set the dataframe index as Smiles (we already verified they are all unique from eachother)
df2 = df1.set_index('nucleophile SMILES')

In [10]:
df2

,reaction,starting electrophile SMILES,input electrophile SMILES,"3,3 Catalyst Substituent",N Catalyst Substituent,solvent,Temperature (Celsius),Temperature (Kelvin),yield (%),ee (%),e.r. 1,e.r. 2,ddG,reference,Link
nucleophile SMILES,,,,,,,,,,,,,,,
C=CC[Si](C)(C)C,1a,O=Cc2ccc1ccccc1c2,O=Cc2ccc1ccccc1c2,c2ccc1ccccc1c2,NS(=O)(=O)C(F)(F)F,toluene,-78,195.15,92.0,92,96.0,4.0,1.231712,"Angew. Chem. Int. Ed. 2016, 55, 13200–13203",https://onlinelibrary.wiley.com/doi/pdfdirect/...
C=CC[Si](C)(C)C,1b,COc2ccc1cc(C=O)ccc1c2,COc2ccc1cc(C=O)ccc1c2,c2ccc1ccccc1c2,NS(=O)(=O)C(F)(F)F,toluene,-78,195.15,80.0,88,94.0,6.0,1.066407,"Angew. Chem. Int. Ed. 2016, 55, 13200–13204",NaN
C=CC[Si](C)(C)C,1c,O=Cc2ccc1cc(Br)ccc1c2,O=Cc2ccc1cc(Br)ccc1c2,c2ccc1ccccc1c2,NS(=O)(=O)C(F)(F)F,toluene,-78,195.15,57.0,86,93.0,7.0,1.002518,"Angew. Chem. Int. Ed. 2016, 55, 13200–13205",NaN
C=CC[Si](C)(C)C,1d,O=Cc1ccccc1,O=Cc1ccccc1,c2ccc1ccccc1c2,NS(=O)(=O)C(F)(F)F,toluene,-78,195.15,85.0,82,91.0,9.0,0.896691,"Angew. Chem. Int. Ed. 2016, 55, 13200–13206",NaN
C=CC[Si](C)(C)C,1e,O=Cc1ccccc1F,O=Cc1ccccc1F,c1ccc3c(c1)CCc2ccccc23,NS(=O)(=O)C(F)(F)F,toluene,-60,213.15,82.0,84,92.0,8.0,1.033884,"Angew. Chem. Int. Ed. 2016, 55, 13200–13207",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C1=CCC=C1,3e,CC/C=C(C)/C=O,CC/C=C(C)/C=O,c3ccc2ccc1ccccc1c2c3,NS(=O)(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F,CH2Cl2,-100,173.15,93.0,96,98.0,2.0,1.338303,"Nat. Commun. 2019, 10, 770",NaN
C1=CCC=C1,3f,O=CC1=CCCCC1,O=CC1=CCCCC1,c3ccc2ccc1ccccc1c2c3,NS(=O)(=O)C(F)(F)C(F)(F)F,CH2Cl2,-100,173.15,93.0,96,98.0,2.0,1.338303,"Nat. Commun. 2019, 10, 770",NaN
C1=CCC=C1,3u,C/C=C(C)/C=O,C/C=C(C)/C=O,c3ccc2ccc1ccccc1c2c3,NS(=O)(=O)C(F)(F)C(F)(F)F,CH2Cl2,-100,173.15,90.0,92,96.0,4.0,1.092856,"Nat. Commun. 2019, 10, 770",NaN


In [11]:
# save to a dictionary
node_data = np.unique(df2.index)

## 5. Compute and Compile Edge Data

In [12]:
# We first need to create subset pairs of the SMILES
smis = [] 
for key in node_data:
    smis.append(key)

from itertools import combinations
smis_subsets = list(combinations(smis,2))
len(smis_subsets)

1176

In [13]:
# create a dictionary, subsets
subsets = {}
for i, (smi1,smi2) in enumerate(smis_subsets):
    field = {}
    field["smi1"] = smi1
    subsets[i] = field
    
    field["smi2"] = smi2
    subsets[i] = field
len(subsets)

1176

In [14]:
# add mol objects to our subsets dictionary
for key,value in subsets.items():
    subsets[key].update({"mol1": Chem.MolFromSmiles(value['smi1'])})
    subsets[key].update({"mol2": Chem.MolFromSmiles(value['smi2'])})

In [15]:
list(subsets.values())[0]

{'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
 'smi2': 'C/C(=C\\CCc1ccccc1)C(=O)/C=C/C(C)C',
 'mol1': <rdkit.Chem.rdchem.Mol at 0x12226b990>,
 'mol2': <rdkit.Chem.rdchem.Mol at 0x12226b220>}

### Compute Tanimoto Similarity (RDKit fingerprint based)

In [16]:
# compute and add Tanimoto Similarity using default RDKit fingerprints
for key,value in subsets.items():
    fp1 = Chem.RDKFingerprint(value['mol1'])
    fp2 = Chem.RDKFingerprint(value['mol2'])
    tan_sim = round(DataStructs.TanimotoSimilarity(fp1,fp2), 3)
    subsets[key].update({"tan_similarity": tan_sim})

In [17]:
list(subsets.values())[0]

{'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
 'smi2': 'C/C(=C\\CCc1ccccc1)C(=O)/C=C/C(C)C',
 'mol1': <rdkit.Chem.rdchem.Mol at 0x12226b990>,
 'mol2': <rdkit.Chem.rdchem.Mol at 0x12226b220>,
 'tan_similarity': 0.489}

### Compute MCS-based Tanimoto Coefficient (multiprocessing)

In [21]:
# get number of processors
import multiprocessing
print(multiprocessing.cpu_count())

4


In [33]:
# From the Python docs, this below is number of usable CPUs (works on Unix/Linux)
# https://docs.python.org/3/library/multiprocessing.html
# we subtracted 2 from total number, so that we can still easily use computer for other tasks
import os
num_cpus = len(os.sched_getaffinity(0)) - 2
num_cpus

In [ ]:
# Add maximum common substructure (MCS)-based Tanimoto Coefficient

#############
#############

def tc_mcs(mol1,mol2,key):
    # get maximum common substructure instance
    mcs = rdFMCS.FindMCS([mol1,mol2],timeout=10) # adding a 10 second timeout
    
    # get number of common bonds
    mcs_bonds = mcs.numBonds
    
    # get number of bonds for each
    # default is only heavy atom bonds
    mol1_bonds = mol1.GetNumBonds()
    mol2_bonds = mol2.GetNumBonds()
    
    # compute MCS-based Tanimoto
    tan_mcs = mcs_bonds / (mol1_bonds + mol2_bonds - mcs_bonds)
    return key, tan_mcs

# create a list of mol1, mol2, and their dictionary key as tuples
mol_tuples = []
for key, value in subsets.items():
    mol_tuples.append((value['mol1'],value['mol2'], key))

# run multiprocessing on the tc_mcs function
from multiprocessing import Pool

if __name__ == '__main__':
    with Pool(num_cpus) as p: # In our case, num_cpus = 14
        star_map = p.starmap(tc_mcs, mol_tuples)
    for key, tan_mcs in star_map:
        subsets[key].update({"tan_mcs": round(tan_mcs,3)})

In [ ]:
list(subsets.values())[0:5]

[{'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
  'smi2': 'C/C(=C\\CCc1ccccc1)C(=O)/C=C/C(C)C',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdc9f50>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdca0a0>,
  'tan_similarity': 0.489,
  'tan_mcs': 0.684},
 {'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
  'smi2': 'C/C=C(C)/C(=O)/C=C/C(C)(C)C',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdca110>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdca180>,
  'tan_similarity': 0.68,
  'tan_mcs': 0.667},
 {'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
  'smi2': 'C/C=C(C)/C(=O)/C=C/C(C)C',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdca2d0>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdc9d20>,
  'tan_similarity': 0.719,
  'tan_mcs': 0.714},
 {'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
  'smi2': 'C/C=C(C)/C=C/CCO',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdca340>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x7fbf5fdca3b0>,
  'tan_similarity': 0.152,
  'tan_mcs': 0.294},
 {'smi1': 'C/C(=C\\CCCCCl)C(=O)/C=C/C(C)C',
  'smi2'

# 6. Save Data

In [ ]:
# Save the subsets data as a pickle
import pickle
with open('../Data/subsets_nucleophile.pickle', 'wb') as outfile:
    pickle.dump(subsets, outfile, pickle.HIGHEST_PROTOCOL)